Upload the Model

In [ ]:
import pickle
from transformers import BertTokenizer

# Specify the path to the saved model
model_path = '/content/final_emotion_model.pkl'

# Load the model
with open(model_path, 'rb') as f:
    model = pickle.load(f)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print("Model loaded successfully.")

Model loaded successfully.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Load the Testing Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rishabhkausish/reddit-depression-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/rishabhkausish/reddit-depression-dataset/versions/1


In [ ]:
import os

path = "/root/.cache/kagglehub/datasets/rishabhkausish/reddit-depression-dataset/versions/1"
files = os.listdir(path)
print(files)

['reddit_depression_dataset.csv']


In [ ]:
import pandas as pd

file_path = os.path.join(path, "reddit_depression_dataset.csv")
data = pd.read_csv(file_path)
data

<ipython-input-5-3b1000ad603b>:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


,Unnamed: 0,subreddit,title,body,upvotes,created_utc,num_comments,label
0,47951,DeepThoughts,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,NaN,0.0
1,47952,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0
2,47957,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0
3,47959,DeepThoughts,"""I want to be like water. I want to slip throu...",NaN,25.0,1.416512e+09,2.0,0.0
4,47960,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0
...,...,...,...,...,...,...,...,...
2470773,10742224,depression,Alone as usual,I just argued with my mom an hour ago; she alw...,9.0,1.672529e+09,6.0,1.0
2470774,10742230,depression,Help needed urgently!,OK basically non of my friend's or family invi...,5.0,1.672530e+09,7.0,1.0
2470775,10742232,depression,I know what your thinking,Don't do It. Not today. Its gonna be ok if you...,10.0,1.672530e+09,8.0,1.0
2470776,10742243,depression,how do I stop having such a cynical view on ma...,"So I don't know, I've recently realized that t...",8.0,1.672531e+09,9.0,1.0


Preprocessing the Dataset

In [ ]:
# Updated preprocess function
import re

def preprocess(text):
    if isinstance(text, list):  # If the entry is a list, join it into a string
        text = ' '.join(text)
    text = text.lower()
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
# Fill NaN and Apply preprocessing to each DataFrame
data['body'] = data['body'].fillna('')  # Replace NaN with an empty string
data['body'] = data['body'].apply(preprocess)

# Drop rows where the 'body' column is NaN
data = data[data['body'].str.strip() != '']
data

<ipython-input-9-3044f50cc748>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['body'] = data['body'].fillna('')  # Replace NaN with an empty string
<ipython-input-9-3044f50cc748>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['body'] = data['body'].apply(preprocess)


,Unnamed: 0,subreddit,title,body,upvotes,created_utc,num_comments,label
0,47951,DeepThoughts,Deep thoughts underdog,only when we start considering ourselves the 9...,4.0,1.405309e+09,NaN,0.0
1,47952,DeepThoughts,"I like this sub, there's only two posts yet I ...",anyway human morality is a joke so long as the...,4.0,1.410568e+09,1.0,0.0
2,47957,DeepThoughts,Rebirth!,hello i am the new guy in charge here besides ...,6.0,1.416458e+09,1.0,0.0
4,47960,DeepThoughts,Who am I?,you could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0
5,47969,DeepThoughts,What is the limit of the knowledge and power a...,personally i think its infinite we will always...,8.0,1.416684e+09,23.0,0.0
...,...,...,...,...,...,...,...,...
2470773,10742224,depression,Alone as usual,i just argued with my mom an hour ago she alwa...,9.0,1.672529e+09,6.0,1.0
2470774,10742230,depression,Help needed urgently!,ok basically non of my friends or family invit...,5.0,1.672530e+09,7.0,1.0
2470775,10742232,depression,I know what your thinking,dont do it not today its gonna be ok if you pu...,10.0,1.672530e+09,8.0,1.0
2470776,10742243,depression,how do I stop having such a cynical view on ma...,so i dont know ive recently realized that the ...,8.0,1.672531e+09,9.0,1.0


In [ ]:
# Function to preprocess a single text entry
def preprocess_text(text, tokenizer, max_length=128):
    return tokenizer(text, truncation=True, padding='max_length', max_length=max_length, return_tensors="pt")


Apply the model to the testing dataset

In [ ]:
import torch
model = model.to(torch.device('cuda'))

def predict_label(text):
    # Preprocess the text
    inputs = preprocess_text(text, tokenizer, max_length=128)

    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Ensure the model is in evaluation mode
    model.eval()

    # Perform inference without gradient calculation
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    return predicted_label



In [ ]:
from google.colab import files
files.download('/content/test_df.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.6/447.6 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 129.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 24.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
# Define a mapping of numeric labels to emotions
label_to_emotion = {
    0: "Happiness",
    1: "Surprise",
    2: "Sadness",
    3: "Anger",
    4: "Disgust",
    5: "Fear",
    6: "Neutral"
}

# Update the predict_label function
def predict_label(text):
    inputs = preprocess_text(text, tokenizer, max_length=128)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_label = torch.argmax(outputs.logits, dim=1).item()
    # Return the emotion as text
    return label_to_emotion[predicted_label]



In [ ]:
# Define Gradio Interface
interface = gr.Interface(
    fn=predict_label,  # Your function
    inputs="text",  # Input type (text field for entering text)
    outputs="text",  # Output type (text output showing prediction)
    title="Emotion Prediction",  # Title of the app
    description="Enter a sentence to predict its emotion using the model.",  # App description
    examples=["I am so happy today!", "I feel so alone right now.","My heart is broken and I don't know how to fix it."]  # Example inputs
)


In [ ]:
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2cf13b0b0d3903fba5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
